## https://kafka-python.readthedocs.io/en/master/apidoc/KafkaConsumer.html#

In [1]:
from kafka import KafkaConsumer
from json import loads

print("Consumer 연결")
consumer = KafkaConsumer(
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='latest' , # 'earliest',
     enable_auto_commit= True ,
     group_id='sr',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

Consumer 연결


In [2]:
consumer.topics()

{'my', 'my-topic', 'my_topic', 'numtest', 'numtest2', 'output'}

In [3]:
consumer.subscribe("numtest")

In [4]:
consumer.subscription()

{'numtest'}

## 발견

### 껏다 켜도 계속 offset(고유한 아이디)계속 유지됨!

* group_id 를 다르게하면?
    * group_id를 다르게하면 offset이 다시 시작!
* 다시 원래 group_id로 바꾸면? 
    * 다시 원래했던것 이후로 진행됨!
* 다른 토픽(numtest2)에서 오게 된다면? (같은 consumer_id / 다른 토픽)
    * message가 보내지지 않음
    * 다시 원래 topic (numtest)로 다시 보내면?
        * 다시 offset을 유지하면서 보내짐!
    
### Producer에서 그냥 send할 때 아이디를 바꾸니 그냥 들어가짐

In [ ]:
for message in consumer:
    print ("topic=%s partition=%d offset=%d: key=%s value=%s" % (message.topic, message.partition,
                                          message.offset, message.key,
                                          message.value))

topic=numtest partition=0 offset=792: key=None value={'1': '1'}
topic=numtest partition=0 offset=793: key=None value={'2': '2'}
topic=numtest partition=0 offset=794: key=None value={'3': '3'}
topic=numtest partition=0 offset=795: key=None value={'4': '4'}
topic=numtest partition=0 offset=796: key=None value={'5': '5'}
topic=numtest partition=0 offset=797: key=None value={'6': '6'}
topic=numtest partition=0 offset=798: key=None value={'7': '7'}
topic=numtest partition=0 offset=799: key=None value={'8': '8'}
topic=numtest partition=0 offset=800: key=None value={'9': '9'}
topic=numtest partition=0 offset=801: key=None value={'10': '10'}
topic=numtest partition=0 offset=802: key=None value={'11': '11'}
topic=numtest partition=0 offset=803: key=None value={'12': '12'}
topic=numtest partition=0 offset=804: key=None value={'13': '13'}
topic=numtest partition=0 offset=805: key=None value={'14': '14'}
topic=numtest partition=0 offset=806: key=None value={'15': '15'}
topic=numtest partition=0 of

topic=numtest partition=0 offset=1682: key=None value={'91': '91'}
topic=numtest partition=0 offset=1683: key=None value={'92': '92'}
topic=numtest partition=0 offset=1684: key=None value={'93': '93'}
topic=numtest partition=0 offset=1685: key=None value={'94': '94'}
topic=numtest partition=0 offset=1686: key=None value={'95': '95'}
topic=numtest partition=0 offset=1687: key=None value={'96': '96'}
topic=numtest partition=0 offset=1688: key=None value={'97': '97'}
topic=numtest partition=0 offset=1689: key=None value={'98': '98'}
topic=numtest partition=0 offset=1690: key=None value={'99': '99'}
topic=numtest partition=0 offset=1691: key=None value={'0': '0'}
topic=numtest partition=0 offset=1692: key=None value={'1': '1'}
topic=numtest partition=0 offset=1693: key=None value={'2': '2'}
topic=numtest partition=0 offset=1694: key=None value={'3': '3'}
topic=numtest partition=0 offset=1695: key=None value={'4': '4'}
topic=numtest partition=0 offset=1696: key=None value={'5': '5'}
topic=n

## MultiConsumer


*  https://kafka-python.readthedocs.io/en/master/usage.html


## https://stackoverflow.com/questions/46491616/python-kafka-multiprocess-vs-thread

In [67]:
from kafka import KafkaConsumer
from json import loads
class KafkaWrapper():
    def __init__(self , group):
        self.consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'],
                                      group_id= group ,
                                      value_deserializer=lambda x: loads(x.decode('utf-8')))
    def consume(self, topic):
        self.consumer.subscribe(topic)
        for message in self.consumer:
            print ("topic=%s partition=%d offset=%d: key=%s value=%s" % (message.topic, message.partition,
                                          message.offset, message.key,
                                          message.value))
            
class ServiceInterface():
    def __init__(self):
        self.kafka_wrapper = KafkaWrapper()

    def start(self, topic):
        self.kafka_wrapper.consume(topic)
class ServiceA(ServiceInterface):
    pass

class ServiceB(ServiceInterface):
    pass

In [68]:
from multiprocessing import Process
def main():
    serviceA = ServiceA()
    serviceB = ServiceB()

    jobs=[]
    # The code works fine if I used threading.Thread here instead of Process
    jobs.append(Process(target=serviceA.start, args=("numtest",)))
    jobs.append(Process(target=serviceB.start, args=("numtest",)))

    for job in jobs:
        job.start()

    for job in jobs:
        job.join()
        
if __name__ == "__main__":
    main()

topic=numtest partition=0 offset=2391: key=None value={'number': '-1.1721794412232729,-2.8434495303559766,-2.4780186666432678,-0.15353940368527141,3.7994969095373134,-1.26741467501264,3.2932817431063284'}
topic=numtest partition=0 offset=2392: key=None value={'number': '2.418759111661469,-0.3053797910655847,0.6390188987519233,-0.252476595449088,1.2542857296649408,-0.11087191221842442,-1.284581100475891'}
topic=numtest partition=0 offset=2393: key=None value={'number': '-1.2999215339990704,-2.580674200808202,0.3905496713901315,0.47006115668818377,-0.8156885876419141,-1.120137014602392,-3.7723102557034527'}
topic=numtest partition=0 offset=2394: key=None value={'number': '-0.9719979764803452,1.1950085106031594,4.1224773705678155,1.4080074000057887,-0.06689012411011727,0.8529028058895654,-2.1047258897081313'}
topic=numtest partition=0 offset=2395: key=None value={'number': '0.5524914294671762,0.3809049442255284,-6.858426751037779,2.7185068729340096,3.7818593937807323,0.9129708056103354,4.

topic=numtest partition=0 offset=2432: key=None value={'number': '0.9883842650833415,0.27175525227518904,0.7250664454677396,-1.674674717949462,-1.3902614124693118,-0.6268654862672335,4.402993100472009'}
topic=numtest partition=0 offset=2433: key=None value={'number': '0.761495942059185,-0.24764019008581115,0.9001467261828155,3.148033081526886,-2.2678021764099294,0.6505188766695884,-0.9032844237380079'}
topic=numtest partition=0 offset=2434: key=None value={'number': '-0.027145842560542862,-1.3230357075450314,-0.10723544399015178,0.3233102023971298,-0.9826732110365531,-3.403997326334082,1.4059935583438763'}
topic=numtest partition=0 offset=2435: key=None value={'number': '4.465676656076265,5.1787250848598045,0.618110969383493,-2.639333981139516,0.03586957375615451,-4.959886176110123,-4.235214903406125'}
topic=numtest partition=0 offset=2436: key=None value={'number': '0.9830607085074095,0.5643165182804899,0.9947158710381102,0.39649797171140255,0.4136984079809396,1.0112885306721615,-1.76

topic=numtest partition=0 offset=2473: key=None value={'number': '-1.4257369424086288,-1.2978151322116445,-0.3699837765586046,0.6565670608149118,-2.6849037695635243,1.8552422784609646,0.8797146910466938'}
topic=numtest partition=0 offset=2474: key=None value={'number': '0.2921080047612905,3.8676693143328453,2.3983221609276497,0.30824230679140685,3.975460301301472,2.7876508374776443,-3.289752448378517'}
topic=numtest partition=0 offset=2475: key=None value={'number': '1.5632872119861325,-1.063321700871199,2.788587890768307,-1.751690316825282,-0.3220194523881114,-0.4907065486241377,-0.16402497334125288'}
topic=numtest partition=0 offset=2476: key=None value={'number': '-1.18297559297418,1.0137915903755021,-4.138421241418767,1.6991936574779436,1.2677919838553688,0.5545673274480417,2.010822374508445'}
topic=numtest partition=0 offset=2477: key=None value={'number': '2.376854307533483,0.686888025407521,-3.046034075202233,2.9944358507665627,3.21052450396922,3.6693025869605473,0.1987713378141

topic=numtest partition=0 offset=2514: key=None value={'number': '0.38006360867262534,-1.15748345683913,-1.3674900197169966,1.4368575514279613,-3.80652256631662,-2.640083940855342,1.0734295789105006'}
topic=numtest partition=0 offset=2515: key=None value={'number': '-3.762602100378467,-2.047633331278757,-3.5900028484725515,-0.040827198777377754,-2.6519189299964148,1.6124833068689952,1.1268043775086598'}
topic=numtest partition=0 offset=2516: key=None value={'number': '-0.8042400380756547,-2.3938948394731754,0.02829851647687355,0.31042172050103867,1.6378585723963521,4.668738269942773,-1.7108412320678854'}
topic=numtest partition=0 offset=2517: key=None value={'number': '0.7030118263893109,2.600655244405826,1.1434054555507356,3.2419276860298076,-1.3159299789536922,-1.3549598544267007,0.766284893380281'}
topic=numtest partition=0 offset=2518: key=None value={'number': '-4.620804625659034,-1.4186319635764164,-2.4690267487079507,0.939725102871583,-0.616409035294302,-1.1381033168232637,-0.65

topic=numtest partition=0 offset=2555: key=None value={'number': '2.1016789929106654,0.43647464934844504,2.5822008914152788,-3.0470516276729356,1.2907316156021518,3.0209629487269165,0.3335115614068234'}
topic=numtest partition=0 offset=2556: key=None value={'number': '4.550653642039893,-0.6859926549017848,-2.121346748879311,-1.798019773180126,-0.6888638985596098,1.5196723379340933,-3.7156394288316013'}
topic=numtest partition=0 offset=2557: key=None value={'number': '0.8688902378607047,-4.736912970458504,3.061070574418463,-0.4845731510462197,-2.649489065889078,-0.623667679131103,-3.301860494710355'}
topic=numtest partition=0 offset=2558: key=None value={'number': '1.7990923788999957,3.108953378642475,0.6851225542970476,-3.674289095235223,-0.766641298118458,-1.7730496918280172,-2.2520131873047218'}
topic=numtest partition=0 offset=2559: key=None value={'number': '2.162700543546723,-0.9830154569510642,-1.0777950511154797,-2.9850372896847284,2.752299676127807,-0.5182771325868211,-0.489326

...

topic=numtest partition=0 offset=2719: key=None value={'number': '0.2806883996569247,-0.895750936512115,-0.8062187220752738,2.331581229568641,2.6226745050310813,1.0019985698165692,-0.3035439911812817'}
topic=numtest partition=0 offset=2720: key=None value={'number': '-1.1727567344775376,-0.40460820364477484,1.243219541765071,-0.6876651422660356,-0.1743442824792133,4.25758354935135,1.898823066530756'}
topic=numtest partition=0 offset=2721: key=None value={'number': '-3.627955494484428,-0.3732173086066937,1.3674020466457804,2.615258895402599,-2.1381328324436253,3.4611080758687454,2.4524971928491976'}
topic=numtest partition=0 offset=2722: key=None value={'number': '-1.4868492352287193,0.6286076838946832,-1.7101829108323268,3.099479003569444,2.1837444975841143,-1.9990722781447405,8.6170246430795'}
topic=numtest partition=0 offset=2723: key=None value={'number': '-0.19879228632561496,-1.0532810851027794,0.933596248898738,-1.2555529086827226,0.3204248765307218,-2.437784203023797,1.911593975

topic=numtest partition=0 offset=2760: key=None value={'number': '0.05062715707388991,-1.9659004289735704,-5.050489014861552,1.9268026707817276,-2.2372486164399317,1.897783264010703,0.24178241852223215'}
topic=numtest partition=0 offset=2761: key=None value={'number': '1.378067238794489,0.7249023426812232,1.932192772410695,-2.8115569756228793,1.615796268574826,-1.7299782648607773,-0.5850495193328287'}
topic=numtest partition=0 offset=2762: key=None value={'number': '-1.374393509214291,-0.19909882829309827,-1.4555750801585174,2.580219670681628,-2.827112916169333,-2.043202512921254,0.8431004460516578'}
topic=numtest partition=0 offset=2763: key=None value={'number': '-1.287943448657659,-0.6180626890083548,-0.13949167145935412,0.7108569743065324,0.4249411734415704,0.9331920290398295,1.5035162933468287'}
topic=numtest partition=0 offset=2764: key=None value={'number': '1.4963948333743518,2.332951742806583,2.0704071108571203,2.5243818959844666,0.17321274087187943,-0.7968418679315166,-1.0185

Process Process-60:
Process Process-59:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/root/anaconda3/envs/kafka/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()


KeyboardInterrupt: 

  File "/root/anaconda3/envs/kafka/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/root/anaconda3/envs/kafka/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/root/anaconda3/envs/kafka/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-67-d99385664870>", line 20, in start
    self.kafka_wrapper.consume(topic)
  File "<ipython-input-67-d99385664870>", line 20, in start
    self.kafka_wrapper.consume(topic)
  File "<ipython-input-67-d99385664870>", line 10, in consume
    for message in self.consumer:
  File "<ipython-input-67-d99385664870>", line 10, in consume
    for message in self.consumer:
  File "/root/anaconda3/envs/kafka/lib/python3.7/site-packages/kafka/consumer/group.py", line 1120, in __next__
    return next(self._iterator)
  File "/root/anaconda3/envs/kafka/lib/python3.7/site-packages/kafka/co

In [86]:
from kafka import KafkaConsumer
from json import loads
from json import dumps
from kafka import KafkaProducer


def on_send_success(record_metadata):
    print("topic : {} , partition : {} , offset : {}".\
          format( record_metadata.topic , record_metadata.partition , record_metadata.offset))

class KafkaWrapper():
    def __init__(self,) :
        self.producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
    
    
    def consume(self, topic , group):
        consumer = KafkaConsumer(bootstrap_servers=['localhost:9092'],
                                      group_id=group,
                                      value_deserializer=lambda x: loads(x.decode('utf-8')))
        consumer.subscribe(topic)
        for message in consumer:
            print ("topic=%s partition=%d offset=%d: key=%s value=%s" % (message.topic, message.partition,
                                          message.offset, message.key,
                                          message.value))
            data = {group : str(message.offset)}
            producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
            producer.send( "result", value = data ).add_callback(on_send_success)
            
class ServiceInterface():
    def __init__(self):
        self.kafka_wrapper = KafkaWrapper()

    def start(self, topic , group):
        self.kafka_wrapper.consume(topic , group)
class ServiceA(ServiceInterface):
    pass

class ServiceB(ServiceInterface):
    pass

In [89]:
from multiprocessing import Process
import threading, logging, time

def main():
    serviceA = ServiceA()
    serviceB = ServiceB()

    jobs=[]
    # The code works fine if I used threading.Thread here instead of Process
    jobs.append(Process(target=serviceA.start, args=("my_topic","SR1",))) # my_topic
    jobs.append(Process(target=serviceB.start, args=("my-topic","SR3",))) # my-topic

    for job in jobs:
        job.start()

    for job in jobs:
        job.join()
if __name__ == "__main__":
    logging.basicConfig(
        format='%(asctime)s.%(msecs)s:%(name)s:%(thread)d:%(levelname)s:%(process)d:%(message)s',
        level=logging.INFO
    )
    main()

2019-08-17 17:29:19,846.846.2638854980469:kafka.conn:139956971476736:INFO:19220:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
2019-08-17 17:29:19,848.848.160982131958:kafka.conn:139956971476736:INFO:19220:Probing node bootstrap-0 broker version
2019-08-17 17:29:19,849.849.9977588653564:kafka.conn:139956971476736:INFO:19220:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: Connection complete.
2019-08-17 17:29:19,955.955.6076526641846:kafka.conn:139956971476736:INFO:19220:Broker version identifed as 1.0.0
2019-08-17 17:29:19,959.959.4199657440186:kafka.conn:139956971476736:INFO:19220:Set configuration api_version=(1, 0, 0) to skip auto check_version requests on startup
2019-08-17 17:29:19,976.976.158618927002:kafka.conn:139956971476736:INFO:19220:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1',

topic=my-topic partition=0 offset=2300: key=None value={'number_2': '6.476247172566793,18.6998425263037,-22.239809251875084,0.8993833695170785,-4.708090694944051'}
topic=my_topic partition=0 offset=2900: key=None value={'number_1': '26.77861074513024,21.489973404180134,-3.2981147820169614,-26.922375719571654,4.044432283062741'}


2019-08-17 17:29:32,831.831.7060470581055:kafka.conn:139956971476736:INFO:2000:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
2019-08-17 17:29:32,831.831.8266868591309:kafka.conn:139956971476736:INFO:1998:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
2019-08-17 17:29:32,833.833.648681640625:kafka.conn:139956971476736:INFO:2000:Probing node bootstrap-0 broker version
2019-08-17 17:29:32,833.833.8930606842041:kafka.conn:139956971476736:INFO:1998:Probing node bootstrap-0 broker version
2019-08-17 17:29:32,835.835.5205059051514:kafka.conn:139956971476736:INFO:2000:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: Connection complete.
2019-08-17 17:29:32,835.835.8886241912842:kafka.conn:139956971476736:INFO:1998:<BrokerConnect

topic=my_topic partition=0 offset=2901: key=None value={'number_1': '51.6498230702154,37.84968414631213,-4.776026913900955,30.629674489653993,-14.178639891725293'}
topic=my-topic partition=0 offset=2301: key=None value={'number_2': '-12.245307968664502,5.292500650077468,4.006699736693014,-12.908338636151022,-2.50952959829298'}


2019-08-17 17:29:32,955.955.4352760314941:kafka.conn:139954520844032:INFO:2000:<BrokerConnection node_id=0 host=ubuntu:9092 <connecting> [IPv4 ('127.0.1.1', 9092)]>: connecting to ubuntu:9092 [('127.0.1.1', 9092) IPv4]
2019-08-17 17:29:32,955.955.3961753845215:kafka.conn:139954520844032:INFO:1998:<BrokerConnection node_id=0 host=ubuntu:9092 <connecting> [IPv4 ('127.0.1.1', 9092)]>: connecting to ubuntu:9092 [('127.0.1.1', 9092) IPv4]
2019-08-17 17:29:33,077.77.07905769348145:kafka.producer.kafka:139956971476736:INFO:2000:Closing the Kafka producer with 0 secs timeout.
2019-08-17 17:29:33,078.78.64522933959961:kafka.conn:139954520844032:INFO:2000:<BrokerConnection node_id=0 host=ubuntu:9092 <connecting> [IPv4 ('127.0.1.1', 9092)]>: Connection complete.
2019-08-17 17:29:33,079.79.18667793273926:kafka.producer.kafka:139956971476736:INFO:2000:Closing the Kafka producer with 0 secs timeout.
2019-08-17 17:29:33,080.80.76667785644531:kafka.conn:139954520844032:INFO:2000:<BrokerConnection node

...

2019-08-17 17:29:46,683.683.2077503204346:kafka.conn:139956881688320:INFO:2000:<BrokerConnection node_id=0 host=ubuntu:9092 <connecting> [IPv4 ('127.0.1.1', 9092)]>: connecting to ubuntu:9092 [('127.0.1.1', 9092) IPv4]
2019-08-17 17:29:46,684.684.9563121795654:kafka.conn:139956881688320:INFO:2000:<BrokerConnection node_id=0 host=ubuntu:9092 <connecting> [IPv4 ('127.0.1.1', 9092)]>: Connection complete.
2019-08-17 17:29:46,686.686.3710880279541:kafka.conn:139956881688320:INFO:2000:<BrokerConnection node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Closing connection. 


topic : result , partition : 0 , offset : 965


Process Process-84:
Process Process-83:
Traceback (most recent call last):


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/root/anaconda3/envs/kafka/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/root/anaconda3/envs/kafka/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/root/anaconda3/envs/kafka/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/root/anaconda3/envs/kafka/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-86-3a7b4b8db603>", line 38, in start
    self.kafka_wrapper.consume(topic , group)
  File "<ipython-input-86-3a7b4b8db603>", line 38, in start
    self.kafka_wrapper.consume(topic , group)
  File "<ipython-input-86-3a7b4b8db603>", line 23, in consume
    for message in consumer:
  File "<ipython-input-86-3a7b4b8db603>", line 23, in consume
    for message in consumer:
  File "/root/anaconda3/envs/kafka/lib/python3.7/site-pac

In [ ]:
from time import sleep
from json import dumps
from kafka import KafkaProducer
import numpy as np

producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         key_serializer = None ,                         
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
def on_send_success(record_metadata):
    print("topic : {} , partition : {} , offset : {}".\
          format( record_metadata.topic , record_metadata.partition , record_metadata.offset))

    
for msg , message in zip(consumer1 , consumer2) :
    print("="*50)
    print ("topic=%s partition=%d offset=%d: key=%s value=%s" % (message.topic, message.partition,
                                          message.offset, message.key,
                                          message.value))
    print ("topic=%s partition=%d offset=%d: key=%s value=%s" % (msg.topic, msg.partition,
                                          msg.offset, msg.key,
                                          msg.value))
    key = str(message.offset) + " " + str(msg.offset)
    producer.send('output', value= {  key : key }  ).add_callback(on_send_success)
    print("="*50)

In [ ]:
# consume earliest available messages, don't commit offsets
KafkaConsumer(auto_offset_reset='earliest', enable_auto_commit=False)

# consume json messages
KafkaConsumer(value_deserializer=lambda m: json.loads(m.decode('ascii')))

# consume msgpack
KafkaConsumer(value_deserializer=msgpack.unpackb)

# StopIteration if no message after 1sec
KafkaConsumer(consumer_timeout_ms=1000)

# Subscribe to a regex topic pattern
consumer = KafkaConsumer()
consumer.subscribe(pattern='^awesome.*')

# Use multiple consumers in parallel w/ 0.9 kafka brokers
# typically you would run each on a different server / process / CPU
consumer1 = KafkaConsumer('my-topic',
                          group_id='my-group',
                          bootstrap_servers='my.server.com')
consumer2 = KafkaConsumer('my-topic',
                          group_id='my-group',
                          bootstrap_servers='my.server.com')